# Часть 1.
----

#### NLP и Трансформеры, знакомство с библиотекой transformers и datasets:

In [1]:
import torch
import numpy as np
import transformers
import warnings
warnings.filterwarnings("ignore")

##### Работа с языковыми моделями и токенизаторами:

In [4]:
from transformers import BertTokenizer
text = "this is an example of using bert transformer tokenizer!"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer(text=text)

{'input_ids': [101, 2023, 2003, 2019, 2742, 1997, 2478, 14324, 10938, 2121, 19204, 17629, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

----
* _input_ids_: айдишник для каждого токена

<br>

* _token_type_ids_: тип токена который разделяет первую и последующие **N** последовательностей в корпусе данных

<br>

* _attention_mask_: маска внимания  - это маска (матрица) из 0 и 1, которая используется для того чтобы определить начало и конец последовательности для модели трансформера, для того чтобы избавить нас от ненужных вычислений. Каждый токенизатор имеет свой собственный способ добавления токенов в первоначальную последовательность. В случае BERT токенизатора, то он просто добавляет **[CLS]** токен в начало и **[SEP]** в конец последовательности (101 и 102 соответственно). Эти числа приходят из айдишников токенов уже натренированного токенизатора.

<br>

##### Экземпляры токенизаторов, могут возвращать вывод в виде pt формата для использования в дальнейшем данных вывода в PyTorch'e либо tf для использования в TensorFlow. Для этого используется аргумент _return_tensors = "pt" (PyTorch tensor) и tf (TensorFlow tensor):_

In [5]:
encoded_input = tokenizer(text, return_tensors="pt") # tf для TensorFlow
print(encoded_input)

{'input_ids': tensor([[  101,  2023,  2003,  2019,  2742,  1997,  2478, 14324, 10938,  2121,
         19204, 17629,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


_Как загружать и использовать модели при помощи Transformers API для целей решения своих задач:_

In [6]:
# https://huggingface.co/models
# https://huggingface.co/docs/transformers/main_classes/model
from transformers import BertModel

In [7]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


__Загрузив себе предобученный трансформер BERT, мы можем его использовать дальше, например подать в него закодированную последовательность, которую получили выше. Такая операция выдаст нам вывод модели в форме **эмбеддингов** и **кросс-аттеншена:**__

In [8]:
output = model(**encoded_input)

In [9]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1012, -0.2307, -0.0572,  ..., -0.2870, -0.0342,  0.9602],
         [-0.4558, -0.3543, -0.3250,  ..., -0.5265,  1.0234,  0.3501],
         [-0.4322, -0.4439, -0.0591,  ..., -0.2053,  0.4723,  0.9452],
         ...,
         [-0.3190, -0.6603,  0.0800,  ..., -0.3020, -0.4188,  0.1921],
         [-0.2751, -0.7012, -0.3172,  ...,  0.4856, -0.3296,  0.3901],
         [ 0.5882,  0.1226, -0.6750,  ...,  0.2499, -0.7004, -0.0285]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.7024, -0.1973,  0.0364,  0.1771,  0.1898, -0.0700,  0.4510,  0.0870,
          0.1515, -0.9997,  0.1503, -0.2038,  0.9657, -0.2926,  0.7801, -0.3612,
          0.3516, -0.3518,  0.2063, -0.0299,  0.3595,  0.9967,  0.6053,  0.1497,
          0.1918, -0.2560, -0.3666,  0.8353,  0.9022,  0.7594, -0.3826,  0.0130,
         -0.9807, -0.2227, -0.5447, -0.9671,  0.1754, -0.6301, -0.0027,  0.0153,
         -0.8641,  0.2278,  0.99

_Для узконаправленных задач таких как заполнение пустого текста при помощи обученных масок используя языковые модели, придумали пайплайнинг, который можно использовать прямо из коробки. Например задача заполненния маски:_

In [10]:
# https://huggingface.co/docs/transformers/main_classes/pipelines
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("The Natural Language Processing is a way of using language models in order to [MASK].")

2022-09-26 19:26:08.831390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.673102855682373,
  'token': 10639,
  'token_str': 'communicate',
  'sequence': 'the natural language processing is a way of using language models in order to communicate.'},
 {'score': 0.06801625341176987,
  'token': 3305,
  'token_str': 'understand',
  'sequence': 'the natural language processing is a way of using language models in order to understand.'},
 {'score': 0.04671889916062355,
  'token': 4553,
  'token_str': 'learn',
  'sequence': 'the natural language processing is a way of using language models in order to learn.'},
 {'score': 0.02921394817531109,
  'token': 3853,
  'token_str': 'function',
  'sequence': 'the natural language processing is a way of using language models in order to function.'},
 {'score': 0.015427297912538052,
  'token': 11835,
  'token_str': 'interact',
  'sequence': 'the natural language processing is a way of using language models in order to interact.'}]

_Для целей, например, последующего анализа текста и мы можем весь вывод из пайплайна обернуть в датафрейм:_

In [11]:
import pandas as pd
pd.DataFrame(unmasker("The Natural Language Processing is a way of using language models in order to [MASK]."))

,score,token,token_str,sequence
0,0.673103,10639,communicate,the natural language processing is a way of us...
1,0.068016,3305,understand,the natural language processing is a way of us...
2,0.046719,4553,learn,the natural language processing is a way of us...
3,0.029214,3853,function,the natural language processing is a way of us...
4,0.015427,11835,interact,the natural language processing is a way of us...


_Если существует возможность воспользоваться пайплайном, который решает уже поставленную задачу - то легче использовать его, так как весь пайплайн (от предобработки до подачи данных) осуществляется за счет уже написанных методов, которые лежат в API Transformers. Допустим нам необходим [zero-shot классификатор](https://en.wikipedia.org/wiki/Zero-shot_learning), мы можем достать его прямо из коробки и использовать преобученную модель уже для своей задачи, не обучая ничего с нуля:_

In [12]:
# https://huggingface.co/google/tapas-base-finetuned-wtq
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

_Попробуем классифицировать предложение на предмет вероятной тематики исходя из контекста:_

In [13]:
sequence_to_classify = "I am going to Belarus in 2023 "
candidate_labels = ['dance', 'fight', 'war']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'I am going to Belarus in 2023 ',
 'labels': ['fight', 'dance', 'war'],
 'scores': [0.6005637049674988, 0.23785468935966492, 0.1615816056728363]}

Поговорим немного про метрики и бенчмаркинг моделей.
___
Существует несколько контрольных показателей для оценки инференса моделей трансформеров (и не только). Benchmarking очень важен для задач, связанных с дообучением в контексте решения мультизадач или использования мультиязыковых моделей в NLP. До этого мы в основном фокусировались на одной единственной метрике качества, которая показывала насколько эффективно и качественно модель работает на этапе инференса. Благодаря API Transformers мы можем дообучать модели решать определенную задачу в контексте дообучения. Но проверка в этот раз будет идти по одному из представленных ниже бенчмарков, так же на эти бенчмарки стоит обращать внимания при поиске и выборе обученной крупной модели, так как в контексте вашей задачи показатели могут быть ниже ожидаемых.

#### **_Наиболее важные сравнительные бенчмарки:_**
----
* ##### GLUE benchmark
* ##### SuperGLUE
* ##### XTREME
* ##### XGLUE
* ##### SQuAD

### [Glue Benchmark:](https://gluebenchmark.com/)
----
Тест GLUE, представляет собой набор инструментов и данных для оценки производительности моделей Multi-Task-Learning (MTL) и трансформеров по конкретному списку задач. Существует лидерборд для мониторинга производительности различных архитектур моделей нейронных сетей. Всего 11 задач в данном бенчмарке. Финальная метрика по любой архитектуре усредняется. Этот бенчмарк влключает в себя 11 задач на контекстное понимание последовательностей:

* Задача __Single-Sentence task__.
* __CoLA:__ (The Corpus of Linguistic Acceptability dataset) - проверка понимания в соответствии с датасетом, составленным из статей по лингвистической теории.
* __SST-2:__ The Stanford Sentiment Treebank dataset. Задача pos/neg классификации на датасете по отзывам фильмов.
* __Датасет и задача по поиску похожих N-грам__ и задачи перефразирования (paraphrase).
* __MRPC (The Microsoft Research Paraphrase Corpus dataset)__ - задача на эквивалентную семантику.
* __QQP (The Quora Question Pairs dataset)__ - задача QA по поиску эквивалентной семантики.
* __STS-B (The Semantic Textual Similarity Benchmark dataset)__ - коллекция датасетов из пар QA по поиску похожих последовательностей.
* __Мини-задачи на проверку инференса моделей.__
* __MNLI (The Multi-Genre Natural Language Inference corpus)__ - коллекция последовательностей, цель задачи - предсказать, влечет ли текст гипотезу (entailment), противоречит гипотезе (contradiction) или ни то, ни другое (neutral).
* __QNLI (Question Natural Language Inference dataset)__ - QA задача и датасет.
* __RTE (The Recognizing Textual Entailment dataset)__ - QA задача и датасет.
* __WNLI (The Winograd Natural Language Inference schema challenge).__

##### __[SuperGLUE benchmark](https://gluebenchmark.com/)__
Как и Glue, SuperGLUE — это новый бенчмарк, созданный с новым набором более сложных задач и данных на проверку понимания языка и предлагающий общедоступный лидерборд по архитектурам моделей. Состоит из 8 языковых задач, метрика также обобщается и приводится к средней.
____
##### __[XTREME benchmark](https://sites.research.google/xtreme)__
General-Purpose mutlilinguistic benchmark. Для мультиязычных и многозадачных моделей.
____
##### __[XGLUE benchmark](https://microsoft.github.io/XGLUE/)__
XGLUE — это еще один межъязыковой эталонный тест для оценки и улучшения производительности кросс-языковых предварительно обученных моделей для понимания естественного языка (NLU) и генерации естественного языка (NLG). Первоначально он состоял из 11 заданий на 19 языках. Основное отличие от XTREME заключается в том, что для каждой задачи тренировочные данные доступны только на английском языке.
____

##### __[SQuAD benchmark](https://rajpurkar.github.io/SQuAD-explorer/)__
SQuAD — это широко используемый набор данных QA.

Прежде чем пойти дальше по основам, нам необходимо рассмотреть такую библиотеку как [**_datasets_**](https://huggingface.co/docs/datasets/index).
----
```
pip install datasets
```

Библиотека datasets является частью экосистемы Transformers API. Библиотека поставляет огромный блок полезных утилит для загрузки, обработки и обмена датасетами через Hugging Face hub. Также библиотека поставляет огромное количество метрик для проверки качества моделей. Стоит иметь в виду, если вы используете публичный датасет для целей вашей задачи, стоит проверить его на предмет доступности, так как информация быстро обновляется и старые данные теряют актуальность и удаляются с репозитория.

In [14]:
from datasets import load_dataset
cola = load_dataset('glue', 'cola')
cola['train'][21:22]

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3 [00:00<?, ?it/s]

{'sentence': ['We yelled ourselves hoarse.'], 'label': [1], 'idx': [21]}

In [15]:
from pprint import pprint
from datasets import list_datasets, list_metrics

all_d = list_datasets()
metrics = list_metrics()

print(f"{len(all_d)} datasets and {len(metrics)} metrics exist in the hub\n")

pprint(all_d[:20], compact=True)
pprint(metrics, compact=True)

10256 datasets and 76 metrics exist in the hub

['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc',
 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue',
 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity',
 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'americas_nli', 'ami',
 'amttl', 'anli']
['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'chrf',
 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'exact_match', 'f1',
 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis',
 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'pearsonr',
 'perplexity', 'poseval', 'precision', 'recall', 'rl_reliability', 'roc_auc',
 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2',
 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s',
 'GMFTBY/dailydialog_evaluate', 'GMFTBY/dailydialogevaluate',
 'Vertaix/vendiscore', 'Vlasta/pr_auc', 'abdu

In [16]:
cola

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [17]:
cola['train'][12]

{'sentence': 'Bill rolled out of the room.', 'label': 1, 'idx': 12}

In [18]:
print("1#",cola["train"].description)

1# GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.




In [19]:
print("2#",cola["train"].citation)

2# @article{warstadt2018neural,
  title={Neural Network Acceptability Judgments},
  author={Warstadt, Alex and Singh, Amanpreet and Bowman, Samuel R},
  journal={arXiv preprint arXiv:1805.12471},
  year={2018}
}
@inproceedings{wang2019glue,
  title={{GLUE}: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding},
  author={Wang, Alex and Singh, Amanpreet and Michael, Julian and Hill, Felix and Levy, Omer and Bowman, Samuel R.},
  note={In the Proceedings of ICLR.},
  year={2019}
}



In [20]:
print("3#",cola["train"].homepage)

3# https://nyu-mll.github.io/CoLA/


_Как было описано выше GLUE бенчмарк - это набор датасетов для проверки решения на 11-ти задачах. Давайте загрузим датасет для MRPC задачи:_

In [21]:
mrpc = load_dataset('glue', 'mrpc')

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

_XTREME (работа с набором данных на разных языках) — еще один популярный набор данных, который мы уже обсуждали. Давайте возьмем и загрузим MLQA из набора XTREME. MLQA.en.de — набора данных для проверки качества на английском и немецком языках, который можно загрузить следующим образом:_

In [22]:
en_de = load_dataset('xtreme', 'MLQA.en.de')

Found cached dataset xtreme (/Users/maratmovlamov/.cache/huggingface/datasets/xtreme/MLQA.en.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
en_de

DatasetDict({
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4517
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 512
    })
})

In [25]:
pd.DataFrame(en_de['test'][0:4])

,id,title,context,question,answers
0,037e8929e7e4d2f949ffbabd10f0f860499ff7c9,Cell culture,An established or immortalized cell line has a...,Woraus besteht die Linie?,"{'answer_start': [31], 'text': ['cell']}"
1,4b36724f3cbde7c287bde512ff09194cbba7f932,Cell culture,The 19th-century English physiologist Sydney R...,Wann hat Roux etwas von seiner Medullarplatte ...,"{'answer_start': [232], 'text': ['1885']}"
2,13e58403df16d88b0e2c665953e89575704942d4,TRIPS Agreement,"After the Uruguay round, the GATT became the b...","Was muss ratifiziert werden, wenn ein Land ger...","{'answer_start': [131], 'text': ['TRIPS']}"
3,d23b5372af1de9425a4ae313c01eb80764c910d8,TRIPS Agreement,"Since TRIPS came into force, it has been subje...",Welche Teile der Welt kritisierten das TRIPS a...,"{'answer_start': [67], 'text': ['developing co..."


#### Data Manipulation с использованием библиотеки datasets:

Датасеты поставляются со множеством нарезок (splits), аргумент split позволяет определить какую часть датасета нам хочется загрузить для работы. В случае если аргумент отсутствует, то вернется датасет - train, test, validation, inference и их комбинаций, то есть всё:

In [26]:
cola_train = load_dataset('glue', 'cola', split ='train')

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Можно указать на какие части разбить датасет и сколько их должно быть при помощи следующего синтаксиса:

In [27]:
cola_sel = load_dataset('glue', 'cola', split = 'train[:300]+validation[-30:]')

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [29]:
cola_sel[0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

Все загруженные датасеты и модели кешируются локально у вас на устройстве, поэтому внимательно следите, чтобы у вас хватало памяти, так как бывают модели и датасеты, которые весият пару десяток гб:

In [30]:
cola_sel.cache_files

[{'filename': '/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/glue-train.arrow'},
 {'filename': '/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/glue-validation.arrow'}]

Data Slicing:

In [32]:
split='train[:100]+validation[:100]'

In [33]:
cola_sel = load_dataset('glue', 'cola', split = split)

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [34]:
cola_sel

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 200
})

In [35]:
# 50% of train and the last 30% of validation
split='train[:50%]+validation[-30%:]'
cola_sel = load_dataset('glue', 'cola', split = split)

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Sorting, indexing, и shuffling

In [36]:
cola_sel.sort('label')['label'][:15]

Loading cached sorted indices for dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-aab7c26ed0bcc7fa.arrow


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [37]:
cola_sel.sort('label')['label'][-15:]

Loading cached sorted indices for dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-aab7c26ed0bcc7fa.arrow


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [38]:
cola_sel[6,19,44]

{'sentence': ['Fred watered the plants flat.',
  'The professor talked us into a stupor.',
  'The trolley rumbled through the tunnel.'],
 'label': [1, 1, 1],
 'idx': [6, 19, 44]}

In [39]:
cola_sel.shuffle(seed=42)[2:5]

Loading cached shuffled indices for dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-35d5d0d68d77c55f.arrow


{'sentence': ['We listened to as little speech as possible.',
  'John not leave.',
  'Which picture of himself does Mary think that John said that Susan likes?'],
 'label': [0, 1, 1],
 'idx': [4169, 3569, 367]}

Применение filter() и map() функций:

In [40]:
cola_sel = load_dataset('glue', 'cola', split='train[:100%]+validation[-30%:]')

Found cached dataset glue (/Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [41]:
cola_sel.filter(lambda s: "kick" in s['sentence'])["sentence"][:3]

Loading cached processed dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fb43a0247d07beca.arrow


['Jill kicked the ball from home plate to third base.',
 'Fred kicked the ball under the porch.',
 'Fred kicked the ball behind the tree.']

In [42]:
cola_sel.filter(lambda s: s['label']== 1)["sentence"][:3]

Loading cached processed dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-6d56199747a2d80f.arrow


["Our friends won't buy this analysis, let alone the next one we propose.",
 "One more pseudo generalization and I'm giving up.",
 "One more pseudo generalization or I'm giving up."]

In [43]:
cola_sel.filter(lambda s: s['label']== cola_sel.features['label'].str2int('acceptable'))["sentence"][:3]

Loading cached processed dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-60a687fadcaa5111.arrow


["Our friends won't buy this analysis, let alone the next one we propose.",
 "One more pseudo generalization and I'm giving up.",
 "One more pseudo generalization or I'm giving up."]

In [44]:
cola_new=cola_sel.map(lambda e:{'len': len(e['sentence'])})
pd.DataFrame(cola_new[0:3])

Loading cached processed dataset at /Users/maratmovlamov/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-50cbe75ae3f27a0e.arrow


,sentence,label,idx,len
0,"Our friends won't buy this analysis, let alone...",1,0,71
1,One more pseudo generalization and I'm giving up.,1,1,49
2,One more pseudo generalization or I'm giving up.,1,2,48


Загрузка локальных файлов (csv, tsv, json, txt и т.д.):

In [45]:
from datasets import load_dataset

data1 = load_dataset('csv', data_files='./../data/data_for_transformers/a.csv', delimiter="\t")

data2 = load_dataset('csv', data_files=['./../data/data_for_transformers/a.csv','./../data/data_for_transformers/b.csv', './../data/data_for_transformers/c.csv'], delimiter="\t")

data3 = load_dataset('csv', data_files={'train':['./../data/data_for_transformers/a.csv','./../data/data_for_transformers/b.csv'], 'test':['./../data/data_for_transformers/c.csv']}, delimiter="\t")

Using custom data configuration default-edec09eebd84f90e
Found cached dataset csv (/Users/maratmovlamov/.cache/huggingface/datasets/csv/default-edec09eebd84f90e/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e06e68d6771ad1b0
Found cached dataset csv (/Users/maratmovlamov/.cache/huggingface/datasets/csv/default-e06e68d6771ad1b0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-0e1bf83aa8a8fc9e
Found cached dataset csv (/Users/maratmovlamov/.cache/huggingface/datasets/csv/default-0e1bf83aa8a8fc9e/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
data_json = load_dataset('json', data_files='a.json')
data_text = load_dataset('text', data_files='a.txt')

Подготовка датасета для работы с моделью трансформеров либо любой другой архитектурой (библиотека не ограничивается функциональностью предназначенной только для 1 архитектуры, можете использовать ее везде, где вам покажется это удобным):

Каждая модель имеет свой собственный способ токенизации. Чтобы, например, воспользоваться токенизатором, мы должны загрузить его из предобученой модели, например _distilBERT-base-uncased model_. Затем при помощи вышеизученного функционала мы можем провести токенизацию и разделить наш датасет на выборки:

In [46]:
from transformers import DistilBertTokenizer

In [47]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

encoded_data3 = data3.map(lambda e: tokenizer( e['sentence'], padding=True, truncation=True, max_length=12), batched=True, batch_size=1000)

Loading cached processed dataset at /Users/maratmovlamov/.cache/huggingface/datasets/csv/default-0e1bf83aa8a8fc9e/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0095c18911536749.arrow
Loading cached processed dataset at /Users/maratmovlamov/.cache/huggingface/datasets/csv/default-0e1bf83aa8a8fc9e/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-da814cdb02354ddc.arrow


In [48]:
data3

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 199
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
})

In [49]:
pprint(encoded_data3['test'][12])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 'input_ids': [101, 2019, 5186, 16010, 2143, 1012, 102, 0, 0, 0, 0, 0],
 'label': 0,
 'sentence': 'an extremely unpleasant film . '}


# Useful links:
[Github Hugginface Intro](https://github.com/huggingface/transformers/tree/main/notebooks)
[Bechmarking and Resource Monitor](https://huggingface.co/docs/transformers/benchmarks)

# Часть 2.
----
### [BERT](https://github.com/google-research/bert)

_BERT (Bidirectional Encoder Representations from Transformers)_  - одна из первых моделей-автокодировщиков, которая использует архитектуру трансформера с небольшими дополнениями (masked language modelling) для целей моделирования языка. Архитектура BERT - многослойный трансформер-энкодер, который мало чем отличается от оригинальной архитектуры трансформера. Сама модель Transformer изначально предназначалась для задач машинного перевода, но основное улучшение, сделанное в BERT, заключается в использовании части архитектуры для языкового моделирования. Такая языковая модель после предварительной подготовки и дообучения способна обеспечить глобальное понимание языка, на котором она обучилась.

----

BERT обучали для решения 2-х глобальных задач:
1. Masked Language Modelling (MLM).
2. Next Sentence Prediction (NSP).


Поговорим про подход Masked Language Modelling который используется в архитектуре BERT.

_Masked Language Modelling_ - это задача тренировки модели на инпуте (последовательность с некоторым количеством токенов на которых была использована маска (какие-то токены в случайном порядке были пропущены)) и получение вывода целого предложения с заполненной маской (пропуском). Почему использование Masked подхода позволяет модели получать наилучшие результаты на инференсе -  если модель может выполнить cloze test (лингвистический тест для оценки понимания языка путем заполнения пропусков), значит, у нее есть общее понимание самого языка. Тогда для других задач нам будет достаточно небольшой дотренировки, чтобы модель показывала результаты не хуже чем на своем бенчмарке.

Пример cloze теста:
```
1. Input:
2. "Deep ____ is the subset of AI."
3. Output:
4. "Deep Learning is the subset of AI."
```

Вторая задача, для которой BERT создавался - задача генерации последовательностей. То есть необходимо было сделать так, чтобы BERT не только учитывал случайные отношения между токенами в корпусе и предсказывал пропущенные, но и также обладал способностью понимать отношения между 2-мя и более предложениями в последовательности.

_Практические задачи, где используется BERT_:
* Text Encoding
* Text Similarity
* Text Summarization
* Text Classification
* Next Sentence (Token) Prediction
* Contextual Understanding
* Question Answering (QA)
* Response Seleciton
* etc.

### [BERT Архитектура:](https://huggingface.co/blog/bert-101)

Существует несколько вариаций архитектур трансформера BERT, но началось все с 2-х главных архитектур, которые затем минимально улучшались:
* BERT-BASE  - модель BERT которая была обучена на относительно большом датасете, архитектура практически не изменялась с оригинальной версией трансформера.
* BERT-LARGE - модель BERT которая была обучена на массивном качественном датасете, когда-то являлась SOTA решением в языковых моделях.
----

BERT - это хорошо обученный стек энкодеров (части архитектуры трансформера), который отвечает за кодирование последовательности. Визуально:

![](./../src/imgs/bert-encoders.png)

Обе базовые версии BERT имеют большое количество слоев энкодера и следующую специфику:

![](./../src/imgs/bert_arch.png)

* 12 слоев энкодера для базовой версии и 24 для большой (также эти слои известны как Transformer Blocks)
* Feed-Forward слои - 768 скрытых нейронов для базовой версии и 1024 для большой
* Attention Heads - 12 для базовой и 16 для большой версии


__Вход для модели трансформера BERT:__

![](./../src/imgs/bert_input.png)

Первым входом (нулевым элементом последовательности) является специальный токен __[CLS]__, который означает начало классификации (CLS - classification). Остальные элементы последовательности - закодированные токены предложения.
Как и обычный энкодер у базового трансформера:
 1. BERT принимает на вход последовательность слов (токенов) и обрабатывает его через первый Encoder.
 2. В Encoder'e последовательность проходит через self-attention блок.
 3. Дальше выход из self-attention'a отдается в Feed-Forward сеть для задачи классификации и результат подается в следующий блок Encoder'a.
 4. Так повторяется до тех пор, пока мы не дойдем до последнего блока Encoder'a

![](./../src/imgs/bert_encoder_blocks.png)




__Выход для модели трансформера BERT:__

![](./../src/imgs/bert_output.png)

Каждый выходной нейрон в модели BERT выдает вектор размерности __hidden_size__ (768 в случае BERT). В случае задач классификации нам инетерес лишь первый токен [CLS], который использует в качестве инпута в классификаторе. При помощи одного единственного классификатора мы можем решать соответственно задачи на классификацию текста, zero-shot классификацию, заполнение маски и т.д. Как это выглядит визуально:

![](./../src/imgs/bert_output2.png)

Вот так кратко выглядит работа трансформера BERT.

### __Представления последовательностей для модели трансформера. [__<u>Модель ELMo</u>__](https://arxiv.org/pdf/1802.05365.pdf):__
----
С какими способами представлений текстовых последовательностей мы уже близко знакомы:
1. CountVectorizer
2. TfIdf
3. Word2Vec
4. Glove
5. Vanilla Embeddings

Будут ли способны все выше указанные представления хорошо понимать смысл (семантическую структуру) последовательности текста?

Не всегда, хорошо было бы иметь такое представление, которое бы учитывало еще и контекст токена, так как в зависимости от контекста токен может обозначать различный смысл.

Например:
1. В 1997 году _**<u>вышла в свет</u>**_ первая книга о мальчике-волшебнике Гарри Поттере.
2. Я  _**<u>вышел</u>**_ на улицу и увидел _**<u>свет</u>**_ горящий где-то в далеке.

Для того чтобы не "хардкодить" такие отношения, придумали новый вариант представления для Embedding'ов, который получил название ELMO - Embeddings for Language Modelling.
Вместо того, чтобы использовать конкретный размер для каждого токена, ELMо берет в рассчет всю последовательность прежде чем закодировать ее в представление Embedding'a. Под капотом работает двунаправленная LSTM рекуррентная нейронная сеть, которая обучается на одной конкретной задаче (узконаправленном корпусе) - составить Embedding для данной задачи.

![](./../src/imgs/elmo_embd.png)

Но мы рассматриваем модель BERT при чем тут ELMo?

Логичный вопрос, но надо было сказать пару слов об ELMo, чтобы понять следующий блок объяснения.

Окей, ELMo реализует механизм контекстуального Embedding'a на уровне предложений, соответственно мы можем смотреть на закодированную последовательность с разных углов и решать разные задачи. Но как запустить такой механизм в BERT трансформере. Все просто, придумали механизм MASKING'a (помним выше я упоминал для каких задач изначально придумывали BERT). В чем его суть:

![](./../src/imgs/bertmasking.png)

Вроде бы разобрались, для тех кто любит посложнее - вот Paper на реализацию трансформера BERT, очень хорошо прокачивает понимание языковых моделей - [PAPER](https://arxiv.org/pdf/1810.04805.pdf)

__**Как и какие задачи можно решать при помощи BERT:**__

![](./../src/imgs/bert_tasks.png)


Независимо от всех этих задач, наиболее важной способностью BERT является его контекстуальное представление текста. Причина, по которой он успешен в различных задачах, заключается в архитектуре кодировщика Transformer, которая представляет ввод в виде неразреженных (плотных) векторов. Эти векторы могут быть легко преобразованы в выходные данные с помощью очень простых классификаторов и использованны дальше в зависимости от цели задачи (например подавать дальше в RNN, CNN и прочие архитектуры).

## Рассмотрим один из вариантов дообучения модели BERT. Будем использовать базу данных отзывов на фильмы [IMDB](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews):

In [74]:
import pandas as pd
imdb_df = pd.read_csv("./../data/imdb/IMDB_Dataset.csv")
reviews = imdb_df.review.to_string(index=None)

# Сохраним как корпус наших данных для дальнейших целей
with open("./../data/imdb/corpus.txt", "w", encoding="utf-8") as file:
    file.writelines(reviews)

Теперь, когда у нас есть собственный корпус данных, нам необходимо дообучить токенизатор (в нашем случае BERT токенизатор, так как для каждого трансформера нужен собственный токенизатор на котором он обучался). BERT изначально обучали на WordPiece токенизации. Как это реализовать:

In [75]:
from tokenizers import BertWordPieceTokenizer

In [76]:
bert_wordpiece_tokenizer = BertWordPieceTokenizer()

In [77]:
bert_wordpiece_tokenizer.train("./../data/imdb/corpus.txt")

Всё :)) Теперь у нас есть собственный дообученный токенизатор, который работает по принципу WordPiece Токенизации. Мы можем посмотреть, что лежит внутри объекта преобученого токенизатора:

In [78]:
bert_wordpiece_tokenizer.get_vocab()

{'ultra': 4246,
 'cure': 5143,
 'dimw': 13308,
 'surp': 5276,
 'strang': 7417,
 'figh': 15304,
 '1917': 16971,
 'cases': 7601,
 '##port': 3568,
 'neigh': 5526,
 '##gic': 2787,
 'il': 4137,
 'gooding': 5868,
 'sandler': 4815,
 'promised': 7531,
 'schizoph': 10476,
 'grem': 10148,
 'nephew': 18196,
 'men': 1433,
 'baby': 2514,
 'coat': 15295,
 'whit': 10001,
 'slipknot': 17943,
 'polit': 2425,
 'ago': 697,
 'animate': 9105,
 'britis': 12541,
 'advert': 3136,
 '##ulse': 7973,
 'match': 5950,
 'tw': 421,
 'judges': 12781,
 'nr': 14038,
 '196': 1721,
 'christ': 1001,
 'dogma': 16232,
 '##orrow': 9969,
 '##olare': 14825,
 '##ison': 1901,
 'gimm': 8661,
 'capable': 10726,
 'mediocrity': 16851,
 'sadly': 2634,
 'fortun': 3523,
 'rudd': 13251,
 'rut': 9830,
 'duckman': 17141,
 'cour': 1757,
 'patri': 5996,
 'gallery': 17273,
 'sho': 1684,
 'retitled': 15832,
 'exceptions': 16164,
 '##ably': 578,
 'jurass': 7719,
 'growing': 3143,
 'shows': 1095,
 '##ness': 1297,
 'ner': 4901,
 'hatred': 8380,
 

Не забываем сохранять преобученный токенизатор, так как в дальнейшем он нам понадобится:

In [79]:
bert_wordpiece_tokenizer.save_model("./../outputs/tokenizer")

['./../outputs/tokenizer/vocab.txt']

In [80]:
# как загрузить модельку токенизатора для BERT которую мы сохранили:
tokenizer = BertWordPieceTokenizer.from_file("./../outputs/tokenizer/vocab.txt")

Теперь, убедившись что теоретически наши токены будут также присутствовать в токенизаторе мы можем с легкостью воспользовать его функционалом:

In [81]:
tokenized_sentence = tokenizer.encode("This is the Natural Language Processing using BERT Transformer.")
tokenized_sentence.tokens

['[CLS]',
 'this',
 'is',
 'the',
 'natural',
 'language',
 'process',
 '##ing',
 'using',
 'bert',
 'transform',
 '##er',
 '.',
 '[SEP]']

Важный момент, окей мы можем дотренировать свой токенизатор, а еще можем воспользоваться BertTokenizerFast из библиотеки transformers и "дополнить" своими данными, чтобы в будущем избежать неловких ситуаций и ошибок во время работы с BERT:

In [82]:
from transformers import BertTokenizerFast #BertTokenizer
tokenizer = BertTokenizerFast.from_pretrained("./../outputs/tokenizer")

Теперь подготовим корпус для быстрого обучения:

In [83]:
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path="./../data/imdb/corpus.txt", block_size=128)

Сортировщик данных (Data Collator) получает данные и подготавливает их к обучению.
Например, возьмем наши данные и подготовим их для моделирования для маскинга языка с вероятностью 0,15.
Цель использования такого механизма — выполнять предварительную обработку «на лету», что позволяет использовать меньше ресурсов. С другой стороны, это замедляет процесс обучения, потому что каждый образец должен быть предварительно обработан на лету во время обучения.

In [84]:
# Также нам необходимо вопсользовать сортировщиком данных для того чтобы смоделировать процесс маскинга
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

Популярная практика задавать аргументы для объекта (можно и не делать, но могут возникнуть ошибки):

In [85]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./../outputs/BERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=128
)

Загрузим BERT трансформер с дефолтными настройками поставки:

In [86]:
from transformers import BertConfig, BertForMaskedLM
bert = BertForMaskedLM(BertConfig())

Создадим объект тренировщика для дотренировки модели:

In [ ]:
BertConfig

In [87]:
from transformers import Trainer

trainer = Trainer(model=bert, args=training_args, data_collator=data_collator, train_dataset=dataset)

Запустим обучение:

In [88]:
trainer.train()

***** Running training *****
  Num examples = 50022
  Num Epochs = 1
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 391


Step,Training Loss



KeyboardInterrupt



Сохраним модельку для будущего использования и возможно доработки:

In [ ]:
trainer.save_model("./../outputs/MyBERT")

Небольшое отступление, мы загрузили дефолтный конфиг для трансформера BERT, но мы также можем указывать свою спецификацию конфига:

In [ ]:
BertConfig()

[Спецификации BERT](https://github.com/google-research/bert)

In [ ]:
tiny_bert_config = BertConfig(max_position_embeddings=512, hidden_size=128,
           num_attention_heads=2,
           num_hidden_layers=2,
           intermediate_size=512)
tiny_bert_config

In [ ]:
tiny_bert = BertForMaskedLM(tiny_bert_config)

In [ ]:
trainer = Trainer(model=tiny_bert, args=training_args,
                     data_collator=data_collator,
                     train_dataset=dataset)

In [ ]:
trainer.train()

Языковая модель BERT также может использоваться в качестве Embedding слоя в любой другой модели глубокого обучения. Например, вы можете загрузить любую предварительно обученную модель BERT или собственную версию, обученную на предыдущем шаге следующим образом:

In [ ]:
# по умолчанию PyTorch имплементация, для TensorFlow TFBertModel и BertTokenizerFast соответсвенно
from transformers import BertModel, BertTokenizerFast
bert = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
# pip install torch-summary
from torchsummary import summary
summary(bert,input_size=(768,),depth=1,batch_dim=1, dtypes=['torch.IntTensor'])

In [ ]:
tokenized_text = tokenizer.tokenize("This is an example of bert tokenizer usage")
tokenized_text

In [44]:
tokenized_text = ["[CLS]"] + tokenized_text + ["[SEP]"]

In [45]:
input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

In [ ]:
bert_output = bert(torch.tensor([input_ids]))
bert_output

### Часть 3:
----
Fine-Tuning модели BERT для классификации текста:

In [89]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
device = "cuda" if torch.cuda.is_available() else "cpu"

Будем использовать вариацию BERT - [DistilBert](https://huggingface.co/docs/transformers/model_doc/distilbert). Настроив "классификационную голову" под себя (будем использовать тот же датасет IMDB - где все 2 класса):

In [90]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path_or_name = "distilbert-base-uncased"

tokenizer = DistilBertTokenizerFast.from_pretrained(model_path_or_name)
model = DistilBertForSequenceClassification.from_pretrained(model_path_or_name, id2label={0:"NEG", 1:"POS"}, label2id={"NEG":0, "POS":1})

https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /Users/maratmovlamov/.cache/huggingface/transformers/tmpgc46pm3c


storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at /Users/maratmovlamov/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /Users/maratmovlamov/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/maratmovlamov/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /Users/maratmovlamov/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at /Users/maratmovlamov/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for /Users/maratmovlamov/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /Users/maratmovlamov/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'v

id2label и label2id отдаются модели на этапе инференса. Существует альтернативный вариант - просто задать их в отдельном конфиге:
```
1. config = AutoConfig.from_pre-trained(....)
2.SequenceClassification.from_pre-trained(.... config=config)
```

Загрузим уже известный нам датасет:

In [94]:
from datasets import load_dataset
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

KeyboardInterrupt: 

In [ ]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

Если у вас мало вычислительных ресурсов, исполните этот кусочек кода, который возьмет малую порцию данных:

In [ ]:
imdb_train= load_dataset('imdb', split="train[:2000]+train[-2000:]")
imdb_test= load_dataset('imdb', split="test[:500]+test[-500:]")
imdb_val= load_dataset('imdb', split="test[500:1000]+test[-1000:-500]")

Проведем данные через токенизатор для того, чтобы подготовить их к обучению:

In [ ]:
enc_train = imdb_train.map(lambda e: tokenizer( e['text'], padding=True, truncation=True), batched=True, batch_size=1000)
enc_test =  imdb_test.map(lambda e: tokenizer( e['text'], padding=True, truncation=True), batched=True, batch_size=1000)
enc_val =   imdb_val.map(lambda e: tokenizer( e['text'], padding=True, truncation=True), batched=True, batch_size=1000)

In [ ]:
import pandas as pd
pd.DataFrame(enc_train)

In [54]:
TrainingArguments?

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./../outputs/MyIMDBModel',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./../outputs/logs',
    logging_steps=200,
    evaluation_strategy= 'steps',
    fp16= torch.cuda.is_available()
)

In [58]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    Precision, Recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': Precision,
        'Recall': Recall
    }

In [59]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=enc_train,
    eval_dataset=enc_val,
    compute_metrics= compute_metrics
)

Using cuda_amp half precision backend


In [ ]:
results=trainer.train()

In [ ]:
q=[trainer.evaluate(eval_dataset=data) for data in [enc_train, enc_val, enc_test]]
pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./../outputs/logs

In [65]:
def get_prediction(text):
    inputs = tokenizer(text, padding=True,truncation=True,
    max_length=250, return_tensors="pt").to(device)
    outputs = model(inputs["input_ids"].to(device),inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

In [ ]:
text = "I didn't like the movie it bored me "
get_prediction(text)[1].item()

In [ ]:
model_save_path = "./../outputs/MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

Можем сложить это все в пайплайн, например для целей production'a:

In [ ]:
from transformers import pipeline, DistilBertForSequenceClassification, DistilBertTokenizerFast

model = DistilBertForSequenceClassification.from_pretrained("./../outputs/MyBestIMDBModel")
tokenizer= DistilBertTokenizerFast.from_pretrained("./../outputs/MyBestIMDBModel")

nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

nlp("the movie was very impressive")

### Дообучения нативными способами PyTorch'a:

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

In [ ]:
model.train()

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-3)

In [ ]:
texts= ["this is a good example","this is a bad example","this is a good one"]
labels= [1,0,1]
labels = torch.tensor(labels).unsqueeze(0)
encoding = tokenizer(texts, return_tensors='pt', padding=True,
truncation=True, max_length=512)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

In [ ]:
outputs

In [ ]:
from torch.nn import functional
labels = torch.tensor([1,0,1])
outputs = model(input_ids, attention_mask=attention_mask)
loss = functional.cross_entropy(outputs.logits, labels)
loss.backward()
optimizer.step()

In [ ]:
loss

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
import datasets
from datasets import load_dataset
sst2= load_dataset("glue","sst2")

from datasets import load_metric
metric = load_metric("glue", "sst2")

In [ ]:
texts=sst2['train']['sentence']
labels=sst2['train']['label']
val_texts=sst2['validation']['sentence']
val_labels=sst2['validation']['label']

In [ ]:
train_dataset= MyDataset(tokenizer(texts, truncation=True, padding=True), labels)
val_dataset=  MyDataset(tokenizer(val_texts, truncation=True, padding=True), val_labels)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader =  DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
from transformers import  AdamW
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(3):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
    model.eval()
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        predictions=outputs.logits.argmax(dim=-1)
        metric.add_batch(
                predictions=predictions,
                references=batch["labels"],
            )
    eval_metric = metric.compute()
    print(f"epoch {epoch}: {eval_metric}")

### Проецирования работы BERT на другие языки. Возьмем для примера турецкий, так как более качественного корпуса с разметкой было не найти, лежит в папке data либо можете скачать с [kaggle](https://www.kaggle.com/savasy/ttc4900):

In [67]:
import pandas as pd
data= pd.read_csv("./../data/turkish_corpora/corpora.csv")
data=data.sample(frac=1.0, random_state=42)

In [ ]:
labels=["teknoloji","ekonomi","saglik","siyaset","kultur","spor","dunya"]
NUM_LABELS= len(labels)
id2label={i:l for i,l in enumerate(labels)}
label2id={l:i for i,l in enumerate(labels)}
data["labels"]=data.category.map(lambda x: label2id[x.strip()])
data.head()

In [ ]:
data.category.value_counts().plot(kind='pie')

In [ ]:
model

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-uncased", max_length=512)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

Разделение данных на подвыборки:

In [ ]:
SIZE= data.shape[0]

train_texts= list(data.text[:SIZE//2])
val_texts=   list(data.text[SIZE//2:(3*SIZE)//4 ])
test_texts=  list(data.text[(3*SIZE)//4:])

train_labels= list(data.labels[:SIZE//2])
val_labels=   list(data.labels[SIZE//2:(3*SIZE)//4])
test_labels=  list(data.labels[(3*SIZE)//4:])

In [ ]:
len(train_texts), len(val_texts), len(test_texts)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = MyDataset(train_encodings, train_labels)
val_dataset = MyDataset(val_encodings, val_labels)
test_dataset = MyDataset(test_encodings, test_labels)

Процесс дообучения при помощи класса Trainer:

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./../outputs/TBERT/',
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory
    logging_dir='./../outputs/multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
     # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
q=[trainer.evaluate(eval_dataset=data) for data in [train_dataset, val_dataset, test_dataset]]
pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs, probs.argmax(),model.config.id2label[probs.argmax().item()]

In [ ]:
text = "Fenerbahçeli futbolcular kısa paslarla hazırlık çalışması yaptılar"
predict(text)

Подготовка к предпродакшену, проверка инференса:

In [ ]:
model_path = "./../outputs/TBERT"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
model_path = "./../outputs/TBERT"
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
nlp("Dolar ve Euro bugün yurtiçi piyasalarda yükseldi")

In [ ]:
nlp("Bayern Münih ile Barcelona bugün karşı karşıya geliyor. Maçı İngiliz hakem James Watts yönetecek!")

### Прочие полезности - задачка суммаризации в несколько строк кода:

In [ ]:
# pip install pdfplumber
import pdfplumber as pp

def get_book_data(book_name):
    all_data = ''
    with pp.open(book_name) as book:
        for page_no, page in enumerate(book.pages, start=1):
            data = page.extract_text()
            all_data += data
    return all_data

In [ ]:
from transformers import pipeline

book_content = get_book_data('./../data/book/example.pdf')
classifier = pipeline("summarization")
summary = classifier(book_content)

In [ ]:
print(summary[0]['summary_text'])